## Ejercicio 2
<p style="text-align:right;" >Armando Olivares</p><br>
Considera los datos usados en clase para el sistema eléctrico español o cualquier otra base de datos con una variable a predecir, al menos otras 5 variables explicativas y un mínimo de 500 observaciones. Queremos ajustar un modelo de regresión lineal mu´ltiple que explique la variable Y en funci´on de las dem´as X (Y = β0X + ) usando “Ridge Regression”:


  \begin{align*}
  \text{minimize}\quad & ||y-X\beta||_2^2 +\rho||\beta||_2^2
\end{align*}


### Electricity market data from esios https://www.esios.ree.es/es

Data description
- price: hourly spot price.
- coal: hourly total production coal plants.
- gas: hourly total production combined cycle plants.
- wind: hourly total production wind plants.
- hidro: hourly total production hidro plants.
- nuclear: hourly total production nuclear plants.
- solar: hourly total production solar plants.
- cogen: hourly total production cogeneration plants.
- demand: hourly total demand in the market.

We want to explain the spot price as a function of the other variables.

In [338]:
%matplotlib inline
import numpy as np
import time
import pandas as pd
import matplotlib.pyplot as plt

price=pd.read_csv('price.csv',delimiter=';',usecols=['value'])
coal=pd.read_csv('coal.csv',delimiter=';',usecols=['value'])
gas=pd.read_csv('gas.csv',delimiter=';',usecols=['value'])
wind=pd.read_csv('wind.csv',delimiter=';',usecols=['value'])
hidro=pd.read_csv('hidro.csv',delimiter=';',usecols=['value'])
nuclear=pd.read_csv('nuclear.csv',delimiter=';',usecols=['value'])
solar=pd.read_csv('solar.csv',delimiter=';',usecols=['value'])
cogen=pd.read_csv('cogen.csv',delimiter=';',usecols=['value'])
demand=pd.read_csv('demand.csv',delimiter=';',usecols=['value'])
df=pd.concat([price, coal, gas, wind, hidro, nuclear, solar, cogen, demand],axis=1)
df.columns=['price', 'coal', 'gas', 'wind', 'hidro', 'nuclear', 'solar', 'cogen', 'demand']

#df=pd.concat([price, demand],axis=1)
#df.columns=['price', 'demand']
normalized_df=(df-df.mean())/df.std()
#normalized_df

**Separamos y Formateamos Nuestro Datos:**

In [339]:
[a,b]=df.shape
ntest=100
df_train = normalized_df.iloc[:(a-ntest), :]
df_test = normalized_df.iloc[(a-ntest):, :]
# train
Y = np.array(df_train['price'])
Y = np.expand_dims(Y, axis=1)
X0 = np.ones([a-ntest,1])
X1 = np.array(df_train[df_train.columns[df_train.columns!='price']])
X = np.concatenate([X0, X1],axis=1)
#test
Y_test = np.array(df_test['price'])
Y_test = np.expand_dims(Y_test, axis=1)
X0_test = np.ones([ntest,1])
X1_test= np.array(df_test[df_test.columns[df_test.columns!='price']])
X_test = np.concatenate([X0_test, X1_test],axis=1)

a) (1 punto) Estimar el valor de los parámetros de la regresián aplicando la solución analítica.

La ecuación para estimar los parametros es de la forma:    $  \\beta_{ls}=(X^T X + \lambda I)^{-1}X^T y$

In [340]:
from numpy.linalg import inv
rho = 1
print (X.T.shape)
time_start = time.clock()
beta_ls_exact=np.dot(inv(np.dot(X.T,X)+ rho*np.identity(X.shape[1])),np.dot(X.T,Y))
time_elapsed = (time.clock() - time_start)
print('time elapsed=',time_elapsed)
print(beta_ls_exact)

(9, 2085)
time elapsed= 0.0007600697936140932
[[-0.0115803 ]
 [ 0.36259694]
 [ 0.21770513]
 [-0.06965771]
 [ 0.36784859]
 [-0.14696702]
 [-0.00694018]
 [ 0.07745367]
 [-0.04210075]]


Calculamos el Error de estos Parámetros:

In [341]:
error = (np.dot(X,beta_ls_exact)- Y)
error_MSE =  np.sum(error*error)/(len(X))
print("MSE: ",error_MSE*100)



MSE:  18.8163444188


Tenenos un Error del **18.8%**, en los datos de **Entrenamiento** veamos si podemos mejorar el modelo

(1.5 puntos) Estimar el valor de los coeﬁcientes de la regresión usando la herramienta minimize del paquete de Python Scipy.optimize. Prueba al menos cuatro solvers diferentes y compara su eﬁciencia en términos de: número de iteraciones totales, número de evaluaciones de la funcón objetivo, gradiente y hesiano, así como el tiempo de computo total

Antes del utilizar el paquete implementamos las formulas del **ridge regression**, es decir la función objetivo, el jacobiano y el hessiano:

In [342]:
from scipy.optimize import minimize

def ridge_reg_brute(beta_lb,X,Y,rho):
    beta_lb=np.matrix(beta_lb)
    z=Y-np.dot(X,beta_lb.T)
    return np.dot(z.T,z)+rho*np.dot(beta_lb, beta_lb.T)

def ridge_reg_der(beta_ls,X,Y, rho):
    beta_ls=np.matrix(beta_ls)
    pp = -2*np.dot((Y-np.dot(X,(beta_ls).T)).T,X) +2*rho*beta_ls
    aa= np.squeeze(np.asarray(pp))
    return aa

def ridge_reg_hess(beta_ls,X,Y, rho):
    ss=2*(np.dot(np.transpose(X),X) + rho*np.identity(X.shape[1]))
    return ss




Obtenidas la ecuaciones, procedemos a utilizar la función *minimize* con el solver **SLSQP**

In [343]:
rho = 2
beta_ls0 = np.zeros(X.shape[1])
time_start = time.clock()
res = minimize(ridge_reg_brute, beta_ls0, args=(X, Y, rho), method='SLSQP',  options={'disp': True})
time_elapsed = (time.clock() - time_start) 
print('time elapsed=',time_elapsed)
print(res.x)

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 393.0171289349598
            Iterations: 12
            Function evaluations: 165
            Gradient evaluations: 12
time elapsed= 0.025782715190871386
[-0.01157493  0.36184416  0.21667032 -0.07063481  0.36681382 -0.14750638
 -0.00769331  0.07728442 -0.03988624]


In [344]:
res.x = res.x.reshape(1,9)
error = (np.dot(X,(res.x).T)- Y)
print(error.shape)
error_MSE =  np.sum(error*error)/(len(X))
print(error_MSE*100)
print('Error=',np.linalg.norm(np.transpose(beta_ls_exact)-res.x,ord=2)/np.linalg.norm(res.x,ord=2))


(2085, 1)
18.8164633569
Error= 0.00522012206213


Con este Modelo obtenemos un **MSE** del **18.81%** muy parecido a la solución análitica, también se puede notar que que la diferencia entres los betas es mínima de alrededor un **0.5%** de error.

Con el Método **Powell:**

In [345]:
rho = 2
beta_ls0 = np.zeros(X.shape[1])
time_start = time.clock()
#run your code
                   
res = minimize(ridge_reg_brute, beta_ls0, args=(X, Y, rho), method='Powell', options={'disp': True})
time_elapsed = (time.clock() - time_start) 
print('time elapsed=',time_elapsed)
print(res.x)


Optimization terminated successfully.
         Current function value: 393.061581
         Iterations: 11
         Function evaluations: 939
time elapsed= 0.14386274793650955
[[-0.01099327  0.35650305  0.22309232 -0.07262772  0.36388209 -0.1489435
  -0.00706378  0.07946284 -0.04327138]]


In [346]:

res.x = res.x.reshape(1,9)
error = (np.dot(X,(res.x).T)- Y)
error_MSE =  np.sum(error.T*error)/(len(X))
print("MSE: ",error_MSE*100)
print('Error=',np.linalg.norm(np.transpose(beta_ls_exact)-res.x,ord=2)/np.linalg.norm(res.x,ord=2))

MSE:  18.81877211
Error= 0.0170556073873


Con el método **Nelder-Mead:**

In [347]:
rho = 2
beta_ls0 = np.zeros(X.shape[1])
time_start = time.clock()
#run your code
                   
res = minimize(ridge_reg_brute, beta_ls0, args=(X, Y, rho), method='Nelder-Mead', options={'disp': True})
time_elapsed = (time.clock() - time_start) 
print('time elapsed=',time_elapsed)
print(res.x)


Optimization terminated successfully.
         Current function value: 414.324787
         Iterations: 1209
         Function evaluations: 1770
time elapsed= 0.17701092831703136
[-0.04609012  0.28978093  0.12053211 -0.151444    0.30599049 -0.27714508
 -0.09662098 -0.03876434  0.18367621]


In [348]:
res.x = res.x.reshape(1,9)
error = (np.dot(X,(res.x).T)- Y)
error_MSE =  np.sum(error*error)/(len(X))
print("MSE: ",error_MSE*100)
print('Error=',np.linalg.norm(np.transpose(beta_ls_exact)-res.x,ord=2)/np.linalg.norm(res.x,ord=2))

MSE:  19.8392151834
Error= 0.585133766434


Observamos como los parámetros direfieren un poco más y el **MSE** asciende hasta **19.83%**, además este método toma **1209** iteraciones.

Con el método **Newton-CG:**

In [349]:
rho = 2
beta_ls0 = np.zeros(X.shape[1])
time_start = time.clock()
res = minimize(ridge_reg_brute, beta_ls0, args=(X, Y, rho), method='Newton-CG', jac=ridge_reg_der, 
               hess=ridge_reg_hess, options={'disp': True,'xtol': 1e-15})
time_elapsed = (time.clock() - time_start) 
print('time elapsed=',time_elapsed)
print(res.x)


Optimization terminated successfully.
         Current function value: 393.017129
         Iterations: 13
         Function evaluations: 14
         Gradient evaluations: 26
         Hessian evaluations: 13
time elapsed= 0.017517852957098512
[-0.01157431  0.36184528  0.21667022 -0.07063424  0.36681453 -0.14750531
 -0.00769267  0.0772854  -0.03988807]


In [350]:
res.x = res.x.reshape(1,9)
error = (np.dot(X,(res.x).T)- Y)
error_MSE =  np.sum(error*error)/(len(X))
print("MSE: ",error_MSE*100)
print('Error=',np.linalg.norm(np.transpose(beta_ls_exact)-res.x,ord=2)/np.linalg.norm(res.x,ord=2))

MSE:  18.8164632451
Error= 0.00521607668832


Más o menos igual que los anteriores en terminos de errores, pero este método tiene un mejor tiempo de ejecución, menos iteraciones y menos evaluaciones del gradiente y del hessiano.

El método **dogleg:**

In [351]:
rho = 2
beta_ls0 = np.zeros(X.shape[1])
time_start = time.clock()
res = minimize(ridge_reg_brute, beta_ls0, args=(X, Y, rho), method='dogleg', jac=ridge_reg_der, 
               hess=ridge_reg_hess, options={'disp': True})
time_elapsed = (time.clock() - time_start) 
print('time elapsed=',time_elapsed)
print(res.x)

Optimization terminated successfully.
         Current function value: 393.017129
         Iterations: 1
         Function evaluations: 2
         Gradient evaluations: 2
         Hessian evaluations: 1
time elapsed= 0.0038792633094999474
[-0.01157431  0.36184528  0.21667022 -0.07063424  0.36681453 -0.14750531
 -0.00769267  0.0772854  -0.03988807]


In [352]:
res.x = res.x.reshape(1,9)
error = (np.dot(X,(res.x).T)- Y)
error_MSE =  np.sum(error*error)/(len(X))
print("MSE: ",error_MSE*100)
print('Error=',np.linalg.norm(np.transpose(beta_ls_exact)-res.x,ord=2)/np.linalg.norm(res.x,ord=2))

MSE:  18.8164632451
Error= 0.00521607668833


Esté método es el **más rápido** de todos, con sola  iteración, 2 evaluaciones del gradiente y  sola evaluación del hessiano, con respecto al **MSE** presenta igual desempeño que los demás, por lo que de elegir un método sería este.

### Si comparamos con el paquete _sklearn_:

In [353]:
from sklearn import linear_model
import matplotlib.pyplot as plt
regr = linear_model.LinearRegression(fit_intercept=False)
regr.fit(X,Y)
print(regr.coef_)
print('Error=',np.linalg.norm(np.transpose(beta_ls_exact)-regr.coef_,ord=2)/np.linalg.norm(regr.coef_,ord=2))

[[-0.01158656  0.36336376  0.21877346 -0.06865408  0.36890455 -0.14641156
  -0.00616682  0.07762911 -0.04437423]]
Error= 0.00533164029906


##  Estimar el valor de los coeﬁcientes de la regresión implementando:


### Método del Gradiente

 $\rightarrow$ From an initial iterate $x_0$

$\rightarrow$ Compute search (descent) directions $p_k=-\nabla f(x_k)$

$\rightarrow$ Far from the solution, compute a steplength $\alpha_k>0$

$\rightarrow$ Movement:
$$x_{k+1} = x_k + \alpha_k\ p_k$$
Until convergence to a local solution

In [354]:
(a,b)=X.shape
beta_lsg=np.zeros(b) #initial value for beta
alpha=0.01
n_iter=50000 #maximim nunber iteration
OF_iter=np.zeros(n_iter)
tol_iter=np.zeros(n_iter)
alpha_iter=np.zeros(n_iter)
i=0;
rho = 1;
tol=10000;
epsilon=1e-2;
time_start = time.clock()
while (i <= n_iter-2) and (tol>epsilon):
    i=i+1
    
    grad=ridge_reg_der(beta_lsg,X,Y, rho)#gradient vector
    ddirect=-grad#descent direction
    ####Armijo rule to adjust alpha########
    sigma = 0.1
    beta= 0.1
    alpha=1
    while (ridge_reg_brute(beta_lsg+alpha*ddirect,X,Y, rho) > 
           ridge_reg_brute(beta_lsg,X,Y, rho)+alpha*sigma*np.dot(grad,ddirect)):
        alpha=alpha*beta
    #######################################
    beta_lsg=beta_lsg+alpha*ddirect
    OF_iter[i]=ridge_reg_brute(beta_lsg, X, Y, rho)
    #tol=np.absolute((OF_iter[i]-OF_iter[i-1])/OF_iter[i]) #tolerance
    tol=np.linalg.norm(grad,ord=2)
    tol_iter[i]=tol
    alpha_iter[i]=alpha

time_elapsed = (time.clock() - time_start) 
print('time elapsed=',time_elapsed)
print('iterations',i)
print(OF_iter[i])
print(beta_lsg)
print(np.transpose(beta_ls_exact))
print('Tolerance=',tol)
print('error=',np.linalg.norm(np.transpose(beta_ls_exact)-beta_lsg,ord=2)/np.linalg.norm(beta_lsg,ord=2))



time elapsed= 0.11513377114897594
iterations 156
392.669371317
[-0.01158019  0.36258754  0.21768264 -0.06967451  0.36783479 -0.1469776
 -0.00695324  0.07744844 -0.04206313]
[[-0.0115803   0.36259694  0.21770513 -0.06965771  0.36784859 -0.14696702
  -0.00694018  0.07745367 -0.04210075]]
Tolerance= 0.00921564942466
error= 8.94886878951e-05


Con este método observamos que se realizaron **156** iteranciones y los coeficientes $ \beta $ son cercanos a la solución exacta y el tiempo de ejecución es mínimo, el **MSE:**

In [355]:
beta_lsg= beta_lsg.reshape(1,9)
error = (np.dot(X,(beta_lsg).T)- Y)
error_MSE =  np.sum(error*error)/(len(X))
print("MSE: ",error_MSE*100)
print('Error=',np.linalg.norm(np.transpose(beta_ls_exact)-beta_lsg,ord=2)/np.linalg.norm(beta_lsg,ord=2))

MSE:  18.8163456334
Error= 8.94886878951e-05


### Con el Método de Newton

In [356]:
(a,b)=X.shape
beta_lsg=np.zeros(b) #initial value for beta
alpha=0.01
n_iter=50000 #maximim nunber iteration
OF_iter=np.zeros(n_iter)
tol_iter=np.zeros(n_iter)
alpha_iter=np.zeros(n_iter)
i=0;
rho = 1;
tol=10000;
epsilon=1e-2;
time_start = time.clock()
while (i <= n_iter-2) and (tol>epsilon):
    i=i+1
    
    grad=ridge_reg_der(beta_lsg,X,Y, rho)#gradient vector
    hess=ridge_reg_hess(beta_lsg,X,Y, rho)
    ddirect=-np.dot(np.linalg.inv(hess),grad)
    ####Armijo rule to adjust alpha########
    sigma = 0.1
    beta= 0.1
    alpha=1
    while (ridge_reg_brute(beta_lsg+alpha*ddirect,X,Y, rho) > 
           ridge_reg_brute(beta_lsg,X,Y, rho)+alpha*sigma*np.dot(grad,ddirect)):
        alpha=alpha*beta
    #######################################
    beta_lsg=beta_lsg+alpha*ddirect
    OF_iter[i]=ridge_reg_brute(beta_lsg, X, Y, rho)
    #tol=np.absolute((OF_iter[i]-OF_iter[i-1])/OF_iter[i]) #tolerance
    tol=np.linalg.norm(grad,ord=2)
    tol_iter[i]=tol
    alpha_iter[i]=alpha

time_elapsed = (time.clock() - time_start) 
print('time elapsed=',time_elapsed)
print('iterations',i)
print(OF_iter[i])
print(beta_lsg)
print(np.transpose(beta_ls_exact))
print('Tolerance=',tol)
print('error=',np.linalg.norm(np.transpose(beta_ls_exact)-beta_lsg,ord=2)/np.linalg.norm(beta_lsg,ord=2))



time elapsed= 0.0010761047742562369
iterations 2
392.6693711
[-0.0115803   0.36259694  0.21770513 -0.06965771  0.36784859 -0.14696702
 -0.00694018  0.07745367 -0.04210075]
[[-0.0115803   0.36259694  0.21770513 -0.06965771  0.36784859 -0.14696702
  -0.00694018  0.07745367 -0.04210075]]
Tolerance= 1.06740173413e-11
error= 3.77558506323e-15


El método de Newton toma 2 iteraciones para converger, con un error despreciable cuando comparamos los coeficientes.

#### Estimar el valor de los coeﬁcientes de la regresión implementando el método del gradiente estocástico

Este método se basa en actualizar los gradientes por cada muestra del dataset, se supone que con pocas iteraciones bastarían para llegar a una solución _aceptable_

In [357]:
(a,b)=X.shape
beta_lsg=np.zeros(b) #initial valuer beta
alpha=1e-2
n_iter=50000 #maximim nunber iteration
OF_iter=np.zeros(n_iter)
tol_iter=np.zeros(n_iter)
tol_iter_aux=np.zeros(len(Y))
alpha_iter=np.zeros(n_iter)
i=0;
rho = 0.2;
grad_old = 0
tol=10000;
epsilon=1e-2;
time_start = time.clock()
while (i <= n_iter-2) and (tol>epsilon):
    i=i+1
 ####Armijo rule to adjust alpha########
    sigma = 0.1
    beta= 0.1

    while (ridge_reg_brute(beta_lsg+alpha*ddirect,X,Y, rho) > ridge_reg_brute(beta_lsg,X,Y, rho)+alpha*sigma*np.dot(grad,ddirect)):    alpha=alpha*beta
   #######################################

    index_t = np.random.permutation(len(Y))
    for index in index_t:
        y = (Y[index])
        x =(X[index,:])
        grad=-2*np.dot((y*np.identity(X.shape[1])-np.dot(x,(beta_lsg).T)).T,x) +2*rho*beta_lsg
        ddirect=-grad#descent direction
        beta_lsg=beta_lsg+alpha*ddirect
        tol_iter_aux[index]=np.linalg.norm(grad,ord=2)

    OF_iter[i]=ridge_reg_brute(beta_lsg, x, y, rho)
    #tol=np.linalg.norm(grad,ord=2)
    tol =np.linalg.norm(grad, ord=2)
    grad_old = grad
    tol_iter[i]=tol
    alpha_iter[i]=alpha   
 


time_elapsed = (time.clock() - time_start) 
print('time elapsed=',time_elapsed)
print('iterations',i)
#print(OF_iter[i])
#print(beta_lsg)
print(np.transpose(beta_ls_exact))
print(beta_lsg)
print('Tolerance=',tol)
print('error=',np.linalg.norm(np.transpose(beta_ls_exact)-beta_lsg,ord=2)/np.linalg.norm(beta_lsg,ord=2))


KeyboardInterrupt: 

Como se Observa este método es algo inestable y presenta alguno problemas de convergencia, pero puede ser muy rapido para millones de datos

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

#plt.plot(OF_iter[1:i])
#plt.figure()
as_fig = plt.plot((np.log(tol_iter)))
plt.show()
#plt.plot(alpha_iter[1:i])
print(tol_iter)

## En resumen:
 * En el análisis de modelos de regresión se deben ajustar convenientement los _hiperparametros_ como $ \lambda, \beta, \rho ,$ etc.
 * En el ejericio notamos que muchos métodos se aproximan a la solución exacta, sin embargo la elección del método depende del también del tiempo de ejecución y del MSE.
 * Con el método de Newton obtuvimos aún mejores resultados y con menos tiempo de ejecución que cuando se compara con alguno métodos de paquetes de _python_.
 * El método del gradiente estocástico es viable cuando se tiene millones de muestras,  es decir en la que priorizamos velocidad en detrimento de exactitud.